In [ ]:
import pandas as pd
import math
import glob
from collections import defaultdict
import os # these two packages are good for searching and navigating file systems
import os.path as op

pd.set_option('display.max_columns', None)

# function to split classification and get rid of _
def split_classification(df):
    df[['domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']] = df.classification.str.split(';', expand=True)
    df['domain'] = df['domain'].str.replace('d__', '') # remove the d__ in front of all observations
    df['phyla'] = df['phyla'].str.replace('p__', '')
    df['class'] = df['class'].str.replace('c__', '')
    df['order'] = df['order'].str.replace('o__', '')
    df['family'] = df['family'].str.replace('f__', '')
    df['genus'] = df['genus'].str.replace('g__', '')
    df['species'] = df['species'].str.replace('s__', '')
    return df

In [ ]:
# for loop: create meta data columns for result data frame
# if data_type = SAG use sag lookup table
# if data_type = MAG use mag lookup table

result = pd.read_csv('/Users/melissaherring/Google Drive/My Drive/MH_project/dramv_full_summary.csv')

sag_cols = ['vir_id','depth', 'classification_via_GTDBTk']
sag_tax = pd.read_csv('~/Documents/Bigelow/Virus_Project/OMZ_MH_Analysis/Data/sag_data/MPvsag_info_230818.csv')[sag_cols]
sag_tax.rename(columns={'classification_via_GTDBTk': 'classification','depth':'sample_depth'}, inplace=True)
sag_tax = split_classification(sag_tax)
sag_tax['virus_name'] = sag_tax['vir_id'].str.replace('vir', 'cv1')

mag_tax = pd.read_csv('~/Documents/Bigelow/Virus_Project/OMZ_MH_Analysis/Data/proximeta_viral_files/vMAG_associations.csv')
mag_cols= ['virus_name', 'sample_depth','domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']
mag_tax = mag_tax[mag_cols]

result[['virus_name','sample_depth','domain','phyla','class','order','family','genus','species']] = []

for index, row in df.iterrows():
    if result['data_type'] == 'vSAG':
        result.merge(sag_tax, on=['ID','virus_name'], how='left')
    else: result.merge(mag_tax, on=['ID','vir_id'], how = 'left')